<a href="https://colab.research.google.com/github/hoafng2511/fuzy_control/blob/main/xecongnghe_iom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install osmnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 1.2 MB/s eta 0:00:00


In [1]:
# ============================================
# APP GỌI XE CÔNG NGHỆ - BẢN HOÀN CHỈNH (OSMnx + Folium + Fuzzy)
# ============================================
import sys, os, math, warnings
warnings.filterwarnings("ignore")

# ---- Cài & import thư viện nếu thiếu
def _pip_install(pkgs):
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

try:
    import osmnx as ox
except ImportError:
    _pip_install(["osmnx"])
    import osmnx as ox

try:
    import networkx as nx
except ImportError:
    _pip_install(["networkx"])
    import networkx as nx

try:
    import folium
except ImportError:
    _pip_install(["folium"])
    import folium

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except ImportError:
    _pip_install(["ipywidgets"])
    import ipywidgets as widgets
    from IPython.display import display, clear_output

try:
    import numpy as np
    import skfuzzy as fuzz
    from skfuzzy import control as ctrl
except ImportError:
    _pip_install(["numpy", "scikit-fuzzy"])
    import numpy as np
    import skfuzzy as fuzz
    from skfuzzy import control as ctrl


# --------------------------------------------
# 1) TẢI MẠNG ĐƯỜNG TP.HCM
# --------------------------------------------
ox.settings.log_console = False
ox.settings.use_cache = True

def load_hcm_graph():
    place = "Ho Chi Minh City, Vietnam"
    return ox.graph_from_place(place, network_type="drive", simplify=True)

G = load_hcm_graph()


# --------------------------------------------
# 2) 20 ĐIỂM MỐC → NODE GẦN NHẤT
# --------------------------------------------
PLACES = {
    "Chợ Bến Thành": (10.772, 106.698),
    "Nhà thờ Đức Bà": (10.779, 106.699),
    "Nhà hát TP.HCM": (10.776, 106.703),
    "Bảo tàng Chứng tích Chiến tranh": (10.779, 106.692),
    "Bitexco Financial Tower": (10.771, 106.704),
    "Landmark 81": (10.795, 106.721),
    "Sân bay Tân Sơn Nhất (Q.Tân Bình)": (10.813, 106.662),
    "BX Miền Đông Mới (TP Thủ Đức)": (10.875, 106.768),
    "BX Miền Tây (Q.Bình Tân)": (10.740, 106.628),
    "KDL Suối Tiên (TP Thủ Đức)": (10.872, 106.803),
    "Công viên Đầm Sen": (10.762, 106.635),
    "Phố đi bộ Bùi Viện": (10.767, 106.694),
    "AEON Mall Tân Phú Celadon": (10.803, 106.615),
    "Vincom Mega Mall Thảo Điền": (10.801, 106.740),
    "Ga Sài Gòn": (10.792, 106.664),
    "Chợ Lớn (Chợ Bình Tây)": (10.754, 106.637),
    "Crescent Mall (Q.7)": (10.728, 106.718),
    "Làng ĐHQG (TP Thủ Đức)": (10.876, 106.772),
    "Thảo Cầm Viên": (10.787, 106.705),
    "KĐT Sala (Thủ Thiêm)": (10.770, 106.731),
}

def build_place_nodes(G, places_dict):
    name_to_node = {}
    for name, (lat, lon) in places_dict.items():
        node = ox.distance.nearest_nodes(G, X=lon, Y=lat)
        name_to_node[name] = node
    return name_to_node

NAME2NODE = build_place_nodes(G, PLACES)


# --------------------------------------------
# 3) HÀM ROUTE + VẼ BẢN ĐỒ FOLIUM
# --------------------------------------------
def shortest_path_and_distance(G, u, v):
    route = nx.shortest_path(G, u, v, weight="length")
    dist_m = nx.shortest_path_length(G, u, v, weight="length")
    return route, dist_m

def draw_route_map(G, route, start_node, end_node, distance_km, fare_kvnd, hour, weather_label, start_name, end_name):
    # Lấy tọa độ
    lat1, lon1 = G.nodes[start_node]["y"], G.nodes[start_node]["x"]
    lat2, lon2 = G.nodes[end_node]["y"], G.nodes[end_node]["x"]
    center = [(lat1 + lat2) / 2, (lon1 + lon2) / 2]

    m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

    # Polyline tuyến đường (màu xanh lá)
    coords = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in route]
    folium.PolyLine(coords, color="green", weight=6, opacity=0.9).add_to(m)

    # Marker Start
    folium.Marker([lat1, lon1], popup=f"Start: {start_name}", tooltip="Start",
                  icon=folium.Icon(color="blue", icon="play", prefix="fa")).add_to(m)
    # Marker End
    folium.Marker([lat2, lon2], popup=f"End: {end_name}", tooltip="End",
                  icon=folium.Icon(color="red", icon="flag", prefix="fa")).add_to(m)

    # Thẻ thông tin
    folium.Marker(
        location=coords[len(coords)//2],
        icon=folium.DivIcon(html=f"""
        <div style="font-size:12px;background:white;padding:6px;border:1px solid #888;border-radius:8px;">
        <b>Quãng đường:</b> {distance_km:.2f} km<br/>
        <b>Giờ đặt:</b> {hour:02d}:00 &nbsp; | &nbsp; <b>Thời tiết:</b> {weather_label}<br/>
        <b>Giá (Fuzzy):</b> {fare_kvnd:.0f}k VND
        </div>""")
    ).add_to(m)
    return m


# --------------------------------------------
# 4) FUZZY PRICING: distance + hour + weather
# --------------------------------------------
# Antecedents
distance = ctrl.Antecedent(np.arange(0, 40.1, 0.1), 'distance')   # km
hour     = ctrl.Antecedent(np.arange(0, 24.01, 0.01), 'hour')     # 0..24
weather  = ctrl.Antecedent(np.arange(0, 10.1, 0.1), 'weather')    # 0..10 (tốt..xấu)

# Consequent (nghìn VND)
fare = ctrl.Consequent(np.arange(0, 800.1, 0.1), 'fare')

# Distance memberships
distance['short']  = fuzz.trimf(distance.universe, [0, 0, 6])
distance['medium'] = fuzz.trimf(distance.universe, [4, 12, 20])
distance['long']   = fuzz.trimf(distance.universe, [15, 40, 40])

# Hour memberships (giờ cao điểm/thấp điểm)
hour['night_low']     = fuzz.trapmf(hour.universe, [0, 0, 4, 6])     # 0-6
hour['morning_peak']  = fuzz.trapmf(hour.universe, [6, 7, 9, 10])    # ~7-9
hour['daytime']       = fuzz.trapmf(hour.universe, [9, 10, 16, 17])  # 10-16
hour['evening_peak']  = fuzz.trapmf(hour.universe, [16, 17, 19, 20]) # ~17-19
hour['late_low']      = fuzz.trapmf(hour.universe, [19, 21, 24, 24]) # 21-24

# Weather memberships (0 tốt → 10 rất xấu)
weather['good']    = fuzz.trapmf(weather.universe, [0, 0, 2.5, 4])
weather['rainy']   = fuzz.trapmf(weather.universe, [3, 4.5, 6.5, 7.5])
weather['bad']     = fuzz.trapmf(weather.universe, [7, 8.5, 10, 10])

# Fare memberships (nghìn VND)
fare['very_low']  = fuzz.trapmf(fare.universe, [30, 40, 70, 90])
fare['low']       = fuzz.trapmf(fare.universe, [80, 100, 140, 170])
fare['medium']    = fuzz.trapmf(fare.universe, [150, 180, 240, 280])
fare['high']      = fuzz.trapmf(fare.universe, [260, 300, 380, 430])
fare['very_high'] = fuzz.trapmf(fare.universe, [400, 450, 520, 600])
fare['surge']     = fuzz.trapmf(fare.universe, [560, 620, 760, 800])

# Một số “toán tử” tiện dụng: peak = morning_peak OR evening_peak
peak = hour['morning_peak'] | hour['evening_peak']
offpeak = hour['night_low'] | hour['daytime'] | hour['late_low']

# Bộ luật:
rules = [
    # Base theo distance + điều kiện thuận lợi
    ctrl.Rule(distance['short']  & offpeak & weather['good'], fare['very_low']),
    ctrl.Rule(distance['short']  & (peak | weather['rainy']), fare['low']),
    ctrl.Rule(distance['short']  & peak & weather['bad'],     fare['medium']),

    ctrl.Rule(distance['medium'] & offpeak & weather['good'], fare['medium']),
    ctrl.Rule(distance['medium'] & (peak | weather['rainy']), fare['high']),
    ctrl.Rule(distance['medium'] & peak & weather['bad'],     fare['very_high']),

    ctrl.Rule(distance['long']   & offpeak & weather['good'], fare['high']),
    ctrl.Rule(distance['long']   & (peak | weather['rainy']), fare['very_high']),
    ctrl.Rule(distance['long']   & peak & weather['bad'],     fare['surge']),
]

fare_ctrl = ctrl.ControlSystem(rules)
fare_sim  = ctrl.ControlSystemSimulation(fare_ctrl)

def fuzzy_fare(distance_km: float, hour_val: float, weather_val: float) -> float:
    """
    Trả về giá (nghìn VND) theo Distance + Hour + Weather.
    - distance_km: 0..40 (sẽ clip)
    - hour_val: 0..24 (giờ trong ngày)
    - weather_val: 0..10 (0 tốt → 10 rất xấu)
    """
    distance_km = float(np.clip(distance_km, 0, 40))
    hour_val    = float(np.clip(hour_val,  0, 24))
    weather_val = float(np.clip(weather_val, 0, 10))

    fare_sim.input['distance'] = distance_km
    fare_sim.input['hour']     = hour_val
    fare_sim.input['weather']  = weather_val
    fare_sim.compute()
    return float(fare_sim.output['fare'])


# --------------------------------------------
# 5) GIAO DIỆN
# --------------------------------------------
start_dd = widgets.Dropdown(
    options=sorted(PLACES.keys()),
    value="Chợ Bến Thành",
    description="Start:",
    layout=widgets.Layout(width="420px")
)
end_dd = widgets.Dropdown(
    options=sorted(PLACES.keys()),
    value="Nhà thờ Đức Bà",
    description="End:",
    layout=widgets.Layout(width="420px")
)

# Giờ đặt (0..23)
hour_slider = widgets.IntSlider(
    value=8, min=0, max=23, step=1, description="Giờ đặt:",
    readout=True, readout_format="d", continuous_update=False
)

# Thời tiết → map sang số 0..10
WEATHER_MAP = {
    "Trời đẹp/khô ráo": 2.0,
    "Mưa nhẹ": 5.5,
    "Mưa to/Ngập/Thời tiết xấu": 8.5,
}
weather_dd = widgets.Dropdown(
    options=list(WEATHER_MAP.keys()),
    value="Trời đẹp/khô ráo",
    description="Thời tiết:",
    layout=widgets.Layout(width="420px")
)

btn = widgets.Button(description="Tính tuyến & Giá", button_style="success", icon="car")
out = widgets.Output()

def on_click_calc(_):
    with out:
        clear_output()
        start_name = start_dd.value
        end_name   = end_dd.value
        if start_name == end_name:
            print("⚠️ Vui lòng chọn 2 điểm khác nhau.")
            return

        u, v = NAME2NODE[start_name], NAME2NODE[end_name]
        route, dist_m = shortest_path_and_distance(G, u, v)
        dist_km = dist_m / 1000.0

        hour_val = int(hour_slider.value)
        weather_label = weather_dd.value
        weather_val = WEATHER_MAP[weather_label]

        # Tính giá fuzzy (nghìn VND)
        price_k = fuzzy_fare(dist_km, hour_val, weather_val)

        # Bản đồ + thông tin
        m = draw_route_map(
            G, route, u, v,
            distance_km=dist_km,
            fare_kvnd=price_k,
            hour=hour_val,
            weather_label=weather_label,
            start_name=start_name,
            end_name=end_name
        )
        display(m)

        print(f"Start: {start_name}  →  End: {end_name}")
        print(f"Quãng đường ước tính: {dist_km:.2f} km")
        print(f"Giờ đặt: {hour_val:02d}:00  |  Thời tiết: {weather_label}")
        print(f"Giá ước tính (Fuzzy): {price_k:.0f} nghìn VND (~ {price_k*1000:,.0f} VND)")

btn.on_click(on_click_calc)

ui = widgets.VBox([
    widgets.HBox([start_dd, end_dd]),
    widgets.HBox([hour_slider, weather_dd]),
    btn,
    out
])

display(ui)


KeyboardInterrupt: 